In [1]:
import os
import numpy as np
import tensorflow as tf

def get_files( file_dir ):
    aluminum = []
    label = []
    for i in range( 12 ):
        for file in os.listdir( file_dir + '/' + str( i ) ):
            if not file.endswith('.jpg') or file.startswith('.'):
                continue  # Skip!
            aluminum.append( file_dir + '/' + str( i ) + '/' + file )
            label.append( i )
    temp = np.array([ aluminum, label ])
    temp = temp.transpose()
    np.random.shuffle( temp )

    aluminum_list = list( temp[:, 0] )
    label_list = list( temp[:, 1] )
    label_list_temp = [int(i) for i in label_list]
    label_list = np.zeros( [len( label_list_temp ), 12] )
    for i in range( len(label_list_temp ) ):
        label_list[i][label_list_temp[i]] = 1
    
    return aluminum_list, label_list

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_batch( image, label, image_width, image_height, batch_size, capacity):
    image = tf.cast( image, tf.string )
    label = tf.cast( label, tf.float32 )
    
    input_queue = tf.train.slice_input_producer( [image, label] )
    
    label = input_queue[1]
    image_contents = tf.read_file( input_queue[0] )
    image = tf.image.decode_jpeg(image_contents, channels=3) 
    image = tf.image.resize_image_with_crop_or_pad(image, image_width
                                                   , image_height)
    image = tf.image.per_image_standardization(image)
    
    image_batch, label_batch = tf.train.batch( [image, label],
                                                                 batch_size = batch_size,
                                                                  num_threads = 1, 
                                                                 capacity = capacity )
    label_batch = tf.reshape(  label_batch, [batch_size, 12] )
    
    return image_batch, label_batch

In [3]:
BATCH_SIZE = 8
CAPACITY = 2100
IMG_W = 384
IMG_H = 384

train_dir = 'guangdong_round1_train2_20180916'
image_list, label_list = get_files( train_dir )
image_batch, label_batch = get_batch( image_list, label_list, IMG_W, IMG_H, BATCH_SIZE, CAPACITY )

In [4]:
def conv_op( input_op, name, kh, kw, n_out, dh, dw, p ):
    n_in = input_op.get_shape()[-1].value

    with tf.name_scope( name ) as scope:
        kernel = tf.get_variable( scope + "w",
            shape = [kh, kw, n_in, n_out], dtype = tf.float32,
            initializer = tf.contrib.layers.xavier_initializer_conv2d() )   
            #Xavier in chapter 4

        conv = tf.nn.conv2d( input_op, kernel, (1, dh, dw, 1), 
                        padding = 'SAME' )
        bias_init_val = tf.constant( 0.0, shape = [n_out], 
                                    dtype = tf.float32 )
        biases = tf.Variable( bias_init_val, trainable = True, name = 'b' )
        z = tf.nn.bias_add( conv, biases )
        activation = tf.nn.relu( z, name = scope )
        p += [kernel, biases]
        return activation

def fc_op( input_op, name, n_out, p ):
    n_in = input_op.get_shape()[-1].value

    with tf.name_scope( name ) as scope:
        kernel = tf.get_variable( scope + "w", 
            shape = [n_in, n_out], dtype = tf.float32,
            initializer = tf.contrib.layers.xavier_initializer() )
        biases = tf.Variable( tf.constant( 0.1, shape = [n_out],
                                        dtype = tf.float32), name = 'b' )
        activation = tf.nn.relu_layer( input_op, kernel, biases, 
                                    name = scope )
        p += [kernel, biases]
        return activation

def mpool_op( input_op, name, kh, kw, dh, dw ):
    return tf.nn.max_pool( input_op, ksize = [1, kh, kw, 1],
                                     strides = [1, kh, kw, 1],
                                     padding = 'SAME',
                                     name = name )

def inference_op( input_op, keep_prob ):
    p = []
    
    #convolution1:
    conv1_1 = conv_op( input_op, name = "conv1_1", kh = 3, kw = 3,
                        n_out = 64, dh = 1, dw = 1, p = p )
    conv1_2 = conv_op( conv1_1, name = "conv1_2", kh = 3, kw = 3,
                        n_out = 64, dh = 1, dw = 1, p = p )
    pool1 = mpool_op( conv1_2, name = "pool1", kh = 2, kw = 2,
                        dh = 2, dw = 2 )

    #convolution2:
    conv2_1 = conv_op( pool1, name = "conv2_1", kh = 3, kw = 3,
                        n_out = 128, dh = 1, dw = 1, p = p )
    conv2_2 = conv_op( conv2_1, name = "conv2_2", kh = 3, kw = 3,
                        n_out = 128, dh = 1, dw = 1, p = p )
    pool2 = mpool_op( conv2_2, name = "pool2", kh = 2, kw = 2,
                        dh = 2, dw = 2 )

    #convolution3
    conv3_1 = conv_op( pool2, name = "conv3_1", kh = 3, kw = 3,
                        n_out = 256, dh = 1, dw = 1, p = p )
    conv3_2 = conv_op( conv3_1, name = "conv3_2", kh = 3, kw = 3,
                        n_out = 256, dh = 1, dw = 1, p = p )
    conv3_3 = conv_op( conv3_2, name = "conv3_3", kh = 3, kw = 3,
                        n_out = 256, dh = 1, dw = 1, p = p )
    pool3 = mpool_op( conv3_3, name = "pool3", kh = 2, kw = 2,
                        dh = 2, dw = 2 )
    
    #convolution4
    conv4_1 = conv_op( pool3, name = "conv4_1", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    conv4_2 = conv_op( conv4_1, name = "conv4_2", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    conv4_3 = conv_op( conv4_2, name = "conv4_3", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    pool4 = mpool_op( conv4_3, name = "pool4", kh = 2, kw = 2,
                        dh = 2, dw = 2 )

    #convolution5
    conv5_1 = conv_op( pool4, name = "conv5_1", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    conv5_2 = conv_op( conv5_1, name = "conv5_2", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    conv5_3 = conv_op( conv5_2, name = "conv5_3", kh = 3, kw = 3,
                        n_out = 512, dh = 1, dw = 1, p = p )
    pool5 = mpool_op( conv5_3, name = "pool5", kh = 2, kw = 2,
                        dh = 2, dw = 2 )
    
    #reshape
    shp = pool5.get_shape()
    flattened_shape = shp[1].value * shp[2].value * shp[3].value
    resh1 = tf.reshape( pool5, [-1, flattened_shape], name = "resh1" )

    #full connection6
    fc6 = fc_op( resh1, name = "fc6", n_out = 1024, p = p )
    fc6_drop = tf.nn.dropout( fc6, keep_prob, name = "fc6_drop" )

    #full conneciton7
    fc7 = fc_op( fc6_drop, name = "fc7", n_out = 1024, p = p )
    fc7_drop = tf.nn.dropout( fc7, keep_prob, name = "fc7_drop" )
    
    fc8 = fc_op( fc7_drop, name = "fc8", n_out = 12, p = p )
    softmax = tf.nn.softmax( fc8, name = "softmax" )
    predictions = tf.argmax( softmax, 1, name = "predictions" )
    return predictions, softmax, fc8, p

In [5]:
x = tf.placeholder( tf.float32, [None, IMG_W, IMG_H, 3], name = "x" )
y_ = tf.placeholder( tf.float32, [None, 12], name = "y_")
keep_prob = tf.placeholder( tf.float32, name = "keep_prob" )
learning_rate = tf.placeholder( tf.float32, name = "learning_rate" )

predictions, softmax, fc8, p = inference_op( x, keep_prob )
cross_entropy = tf.reduce_mean( -tf.reduce_sum( y_ * tf.log(softmax), reduction_indices = [1] ) )
train_step = tf.train.AdamOptimizer( learning_rate ).minimize( cross_entropy )
correct_prediction = tf.equal( predictions, tf.argmax( y_, 1 ) )

accuracy = tf.reduce_mean( tf.cast( correct_prediction, tf.float32 ) )

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    i = 0

    sess.run( tf.global_variables_initializer() )
    sess.run( tf.local_variables_initializer() )


    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners( coord=coord )

    try:
        while not coord.should_stop() and i <= 10000:
            img, label = sess.run( [image_batch, label_batch] )


            if i % 10 == 0:
                train_accuracy = accuracy.eval( feed_dict = 
                                               {x: img, y_: label, keep_prob: 1.0} )
                print( "step %d, training accuracy %g"%( i, train_accuracy ) )
            
            if i % 1000 == 0:
                saver.save(sess, 'model/')
            train_step.run( feed_dict = {x: img, y_: label, keep_prob: 0.5, learning_rate: 1e-4} )

            i += 1

    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
    coord.join(threads)